# Process ROH Results into one big dataframe
Contains cleaning lines (i.e. to remove duplicates), fix flipped coordinates

In [9]:
import numpy as np
import os  # For Saving to Folder
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colorbar as clb
import matplotlib.colors as cls
from matplotlib import gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.basemap import Basemap
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

import socket
import os as os
import sys as sys
import multiprocessing as mp

socket_name = socket.gethostname()
print(socket_name)
if socket_name == "VioletQueen":
    path = "/home/harald/git/HAPSBURG/"   # The Path on Harald's machine
elif socket_name.startswith("midway2"):
    print("Midway jnovmbre partition detected.")
    path = "/project2/jnovembre/hringbauer/HAPSBURG/"  # The Path on Midway Cluster
else: 
    raise RuntimeWarning("Not compatible machine. Check!!")
    
os.chdir(path)  # Set the right Path (in line with Atom default)
print(os.getcwd()) # Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns
print(f"CPU Count: {mp.cpu_count()}")

### Additional Imports from Support Packages
sys.path.append("./package/hapsburg/")
from PackagesSupport.pp_individual_roh_csvs import extract_sub_df_geo_kw, give_df_clsts

midway2-0401.rcc.local
Midway jnovmbre partition detected.
/project2/jnovembre/hringbauer/HAPSBURG
CPU Count: 28


### Functions that pre-process Data
Add "region" Field. Add "color" (based on Time) field

In [10]:
############################################################################
### Post-Process Regions
def set_regions_from_csv(df, csv_path = "./Data/RegionDefinition/regions.csv", 
                         output=True, sep=","):
    """Set Region column in df, by loading coordinates from csv_path"""
    df_regions= pd.read_csv(csv_path, sep=sep)
    for index, row in df_regions.iterrows():
        region = row["Region"] 
        if output:
            print(f"Doing {region}...")
        kw = str(row["Keywords"]).split("|") # produce list from Keywords
        df_t = extract_sub_df_geo_kw(df, row["Lat_low"], row["Lat_high"], row["Lon_low"], 
                                     row["Lon_high"], kw, output=output)
        idx = df["iid"].isin(df_t["iid"]) # Get Indices of Sub Dataframe
        df.loc[idx, "region"] = row["Region"] 
    return df

############################################################################
### Post-Process Colors
def set_colors_from_csv(df, csv_path = "./Data/RegionDefinition/colors.csv", 
                         output=True, sep=","):
    """Set Color column in df, by loading colors from csv_path"""
    df_colors= pd.read_csv(csv_path, sep=sep)
    for index, row in df_colors.iterrows():
        color = row["Color"] 
        ig = row["InternalGroup"]
        kw = str(row["Keywords"]).split("|") # produce list from Keywords
        df_t = give_df_clsts(df, search=kw, col="pop")
        idx = df["iid"].isin(df_t["iid"]) # Get Indices of Sub Dataframe
        df.loc[idx, "color"] = color
        df.loc[idx, "clst"] = row["clst"]
        
        if output:
            print(f"Doing {ig}...")
            print(f"Found {np.sum(idx)} Inds - set to color: {color}")
        
    ### Do old HunterGatherers
    return df

def set_color_hg_minage(df, color="blue", min_age=10500, output=True):
    """Set the color for all ancient Huntergatherers."""
    idx = df["age"] > min_age
    df.loc[idx, "color"] = color
    if output:
        print(f"Found {np.sum(idx)} Inds >{min_age} BP - set to color: {color}")
    return df
    
def set_color_modern(df, color="white", output=True):
    """Set color for all Modern Samples"""
    idx = df["age"] == 0
    df.loc[idx, "color"] = color
    df.loc[idx, "clst"] = "Modern"
    if output:
        print(f"Found {np.sum(idx)} Moderns - set to color: {color}")
    return df

def remove_ids(df, csv_path = "./Data/RegionDefinition/remove_ids.csv", output=True, del_col="iid"):
    """Remove Individuals whose del_col column contains
    string from del_strings (list)"""
    del_list = np.loadtxt(csv_path, dtype="str")
    
    n=len(df)
    for ds in del_list:
        df = df[~df[del_col].str.contains(ds)]
    if output:
        print(f"Removed {n-len(df)} / {n} Individuals in Deletion List.")
    return df

def remove_duplicates(df, cov_col="n_cov_snp", id_col="iid", master_col = "Master ID",
                      path_master="./Data/ReichLabEigenstrat/Raw.v42.4/v42.4.1240K.anno",
                      output=True):
    """Remove duplicates based on merging with Master Dataframe.
    Return Filtered Dataframe
    id_col: Column onto which to merge
    """
    n = len(df)
    df_meta = pd.read_csv(path_master, sep="\t")
    df_meta[id_col] = df_meta.filter(regex='Instance ID')
    
    df_meta = df_meta[[id_col, master_col]]  # Only relevant columns
    df_merge = pd.merge(df, df_meta, on=id_col, how="left")  # Merge on IID
    df_merge = df_merge.sort_values(by=cov_col, ascending=False) # Put IIDs with most SNPs first
    ### Fill up NaNs with IDs
    idx = df_merge[master_col].isnull()
    df_merge.loc[idx, master_col] = df_merge.loc[idx, id_col]
    df_merge = df_merge.drop_duplicates(subset=master_col, keep="first")
    
    df_merge = df_merge.drop(columns=master_col)  #Drop the Master ID Col again

    if output:
        print(f"Removed {n- len(df_merge)} / {n} Duplicates")
    return df_merge

############################################################################
############################################################################

def merge_in_economy_iid(df, path_economy="", 
                         economy_col="economy",
                         match_col = "iid", 
                         case=False):
    """Create/Set Column economy_col into dataframe df. Check for substring matches (to be future proof)
    Return modified dataframe.
    match_col: What columns to match
    economy_col: What column to transfer over
    case: Whether IID substring matching is case sensitive"""
    df_match = pd.read_csv(path_economy)  # Load the data
    
    ### Create new, empty economy column if needed
    if not economy_col in df.columns:
        df[economy_col] = np.nan
    
    ### Match all IIDs
    for i,m in enumerate(df_match[match_col]):
        m = m.rstrip()  # Remove all right whitespaces
        if len(m)==0:
            continue
        idx = df[match_col].str.contains(m, case=case)
        df.loc[idx, economy_col] = df_match.loc[i,economy_col]
    return df

def set_economy_color(df, path_color_df="./Data/RegionDefinition/economy_colors.csv", 
                      color_col="color", economy_col="economy"):
    """Set Color Based on Economy.
    Assume color column in df exists"""
    df_c = pd.read_csv(path_color_df)
    dct = dict(zip(df_c[economy_col], df_c[color_col]))  # Create mapping dictionary
    df[color_col] = df[economy_col].map(dct).fillna(df[color_col])  # Only Map hits
    return df

def set_latlon_from_csv(df, csv_path = "./Data/Coordinates/geo_clst.csv", 
                        output=True, sep="\t", col="pop", col_match="clst"):
    """Set Color column in df, by loading colors from csv_path"""
    df_geo = pd.read_csv(csv_path, sep=sep)

    for index, row in df_geo.iterrows():
        lat = row["lat"] 
        lon = row["lon"]
        kw = row[col_match]
        df_t = df[df[col].str.contains(kw)]
        idx = df["iid"].isin(df_t["iid"]) # Get Indices of Sub Dataframe
        df.loc[idx, "lat"] = lat
        df.loc[idx, "lon"] = lon

        if output:
            print(f"Doing {kw}...")
            print(f"Found {np.sum(idx)} Inds - set GPS")
    ### Do old HunterGatherers
    return df

def set_islands(df, df_isl, col="region", max_d=2, new_val="Islands", output=True):
    """Set col in dataframe df that are close to df_isl lat lon
    to new new_val. Return modified dataframe
    max_d: Maximum Distance (in latlon space)"""
    for index, row in df_isl.iterrows():
        lat_close = (np.abs(df["lat"] - row["lat"]) < max_d)
        lon_close = (np.abs(df["lon"] - row["lon"]) < max_d)
        idx = (lat_close & lon_close)
        
        if output:
            region1 = row["clst"]
            print(f"{region1}: {np.sum(idx)}")
        df.loc[idx, col] = new_val
    return df

# Load all varying Dataframes

In [11]:
### Reich Dataframe
# Define Individuals we want to delete (Duplicates/Neanderthals)
df_r = pd.read_csv("./Empirical/Eigenstrat/Reichall/combined_roh_v42.csv", sep="\t")
df_r['region'] = "all"   # Place Holder
print(f"Loaded Reich Data: {len(df_r)}")
cols = df_r.columns # Extract key column names in right order

### Sardinians from Marcus et all
df_sard = pd.read_csv("./Empirical/1240k/MarcusAncs/combined_roh05.csv", sep="\t")
df_sard = df_sard[df_sard["pop"].str.contains("Sar-")]  #Extract Sardinia Data
df_sard["region"]="Sardinia"
df_sard = df_sard[cols]
print(f"Loaded Sardinian Data: {len(df_sard)}")

### Human Origin Data
df_ho = pd.read_csv("./Empirical/HO/combined_roh05.csv", sep="\t")
df_ho["region"] = df_ho["pop"] # Will be later overwritten for Macro Region!
df_ho["color"] = "gray"
df_ho = df_ho[cols]
print(f"Loaded modern Data: {len(df_ho)} Individuals")

### Concatenate the Dataframes
df_all = pd.concat([df_r, df_sard, df_ho])
print(f"Concatenated {len(df_all)} Individual ROH Data!")

### Filter to good individuals
df_all =df_all[df_all["include_alt"]>0] 
print(f"Filtered to {len(df_all)} Individuals with include_alt>0")

Loaded Reich Data: 1923
Loaded Sardinian Data: 40
Loaded modern Data: 1941 Individuals
Concatenated 3904 Individual ROH Data!
Filtered to 3811 Individuals with include_alt>0


### Remove Individuals in Deletion List and also Duplicates 
(based on master ID)

In [12]:
csv_path = "./Data/RegionDefinition/remove_ids.csv"
df_all = remove_ids(df_all, csv_path)
df_all = remove_duplicates(df_all, 
                           path_master="./Data/ReichLabEigenstrat/Raw.v42.4/v42.4.1240K.anno")

Removed 21 / 3811 Individuals in Deletion List.
Removed 58 / 3790 Duplicates


### Merge in Coordinats

### Fill in missing coordinates from outside source

In [13]:
df_geo = pd.read_csv("./Data/Coordinates/MittnikNatComm2018_Coordinates.csv", sep="\t")
df_geo.index = df_geo["iid"]
df_all.index = df_all["iid"]
df_all.update(df_geo)

### Merge in group GPS
df_all = set_latlon_from_csv(df_all, csv_path = "./Data/RegionDefinition/geo_clst.csv", col="clst",
                    output=False, sep="\t")

### Merge in individuals GPS
df_all = set_latlon_from_csv(df_all, csv_path = "./Data/RegionDefinition/geo_iid.csv", 
                             col="iid", col_match="iid",output=False, sep="\t")

### Set the regions from .csv
csv_path = "./Data/RegionDefinition/regions.csv"
df_t = set_regions_from_csv(df_all, csv_path)

### Set the Islands
df_isl = pd.read_csv("./Data/RegionDefinition/islands_gps.csv", sep="\t")
df_t = set_islands(df_t, df_isl, col="region", new_val="Islands")

Doing Iberia...
Found 244 Individuals; 214 from Geography
Doing Balkans...
Found 149 Individuals; 149 from Geography
Doing Aegan...
Found 126 Individuals; 126 from Geography
Doing Central Europe...
Found 191 Individuals; 184 from Geography
Doing Black Sea...
Found 32 Individuals; 28 from Geography
Doing North Africa...
Found 56 Individuals; 55 from Geography
Doing Britain...
Found 166 Individuals; 162 from Geography
Doing Baltic Sea...
Found 117 Individuals; 117 from Geography
Doing Sardinia...
Found 76 Individuals; 76 from Geography
Doing Levante...
Found 204 Individuals; 203 from Geography
Doing Eastern Europe...
Found 141 Individuals; 141 from Geography
Doing Central Italy...
Found 135 Individuals; 135 from Geography
Doing Steppe...
Found 547 Individuals; 547 from Geography
Doing Patagonia...
Found 10 Individuals; 10 from Geography
Doing Andean...
Found 39 Individuals; 39 from Geography
Doing Pacific NW...
Found 29 Individuals; 29 from Geography
Doing Atlantic Coast...
Found 22 Indi

### Set the colors

### Set the Economies (Mode of Food Production)

In [14]:
df_t["color"]= "silver" # Make Tabula Rasa
csv_path = "./Data/RegionDefinition/colors.csv"
#df_t = set_colors_from_csv(df_t, csv_path)
#df_t = set_color_hg_minage(df_t, color="purple")
df_t = set_color_modern(df_t, color="yellow")

### Set it based on Food Economy
df_t = merge_in_economy_iid(df_t, path_economy="./Data/RegionDefinition/economy_clst.csv", match_col='clst')   # Do the Individual Matches (overwriting)
df_t = merge_in_economy_iid(df_t, path_economy="./Data/RegionDefinition/economy_iid.csv", match_col='iid')   # Do the Individual Matches (overwriting)
df_t = set_economy_color(df_t, path_color_df="./Data/RegionDefinition/economy_colors.csv")

Found 1934 Moderns - set to color: yellow


### Save the Summary Dataframe

In [15]:
savepath="./Empirical/roh_all_inds_final_v42.csv"
if len(savepath)>0:
    df_t.to_csv(savepath, sep="\t", index=False)
    print(f"Saved {len(df_all)} Individual ROH to: {savepath}")

Saved 3732 Individual ROH to: ./Empirical/roh_all_inds_final_v42.csv


# Area 51

In [22]:
give_df_clsts(df_t, search=["Himalaya"], col="region")

,iid,pop,max_roh,sum_roh>4,n_roh>4,sum_roh>8,n_roh>8,sum_roh>12,n_roh>12,sum_roh>20,...,lon,age,study,clst,mean_cov,n_cov_snp,include_alt,region,color,economy
iid,,,,,,,,,,,,,,,,,,,,,
C1.SG,C1.SG,Nepal_Chokhopani_2700BP.SG,13.058001,61.822600,7,45.668692,4,26.106102,2,0.000000,...,83.982202,2775.0,JeongPNAS2016,Nepal_Chokhopani_2700BP.SG,7.182,1178495,1,Himalaya,gray,NaN
S35.SG,S35.SG,Nepal_Samzdong_1500BP.SG,17.873500,55.456002,7,17.873500,1,17.873500,1,0.000000,...,83.982202,1500.0,JeongPNAS2016,Nepal_Samzdong_1500BP.SG,3.720,1135160,1,Himalaya,gray,NaN
S10.SG,S10.SG,Nepal_Samdzong_1500BP.SG,19.573301,57.669193,6,33.694798,2,33.694798,2,0.000000,...,83.982202,1500.0,JeongPNAS2016,Nepal_Samdzong_1500BP.SG,3.263,1124932,1,Himalaya,gray,NaN
M63.SG,M63.SG,Nepal_Mebrak_2125BP.SG,22.859901,43.045801,4,33.257801,2,22.859901,1,22.859901,...,83.982202,2125.0,JeongPNAS2016,Nepal_Mebrak_2125BP.SG,1.024,750057,1,Himalaya,gray,NaN
Kusunda_9,Kusunda_9,Kusunda,4.394400,4.394400,1,0.000000,0,0.000000,0,0.000000,...,82.500000,0.0,Lazaridis et al. 2014,Modern,NaN,554869,1,Himalaya,gray,NaN
Kusunda_8,Kusunda_8,Kusunda,17.500895,363.965840,49,149.528624,13,58.668607,4,0.000000,...,82.500000,0.0,Lazaridis et al. 2014,Modern,NaN,554199,1,Himalaya,gray,NaN
Kusunda_7,Kusunda_7,Kusunda,0.000000,0.000000,0,0.000000,0,0.000000,0,0.000000,...,82.500000,0.0,Lazaridis et al. 2014,Modern,NaN,554051,1,Himalaya,gray,NaN
Kusunda_6,Kusunda_6,Kusunda,8.230602,15.890103,2,8.230602,1,0.000000,0,0.000000,...,82.500000,0.0,Lazaridis et al. 2014,Modern,NaN,553906,1,Himalaya,gray,NaN
Kusunda_5,Kusunda_5,Kusunda,0.000000,0.000000,0,0.000000,0,0.000000,0,0.000000,...,82.500000,0.0,Lazaridis et al. 2014,Modern,NaN,553470,1,Himalaya,gray,NaN


In [47]:
df_tt[df_tt["clst"]=="Russia_Bolshoy"]

,iid,pop,max_roh,sum_roh>4,n_roh>4,sum_roh>8,n_roh>8,sum_roh>12,n_roh>12,sum_roh>20,...,lon,age,study,clst,mean_cov,n_cov_snp,include_alt,region,color,economy
iid,,,,,,,,,,,,,,,,,,,,,
BOO004.A0101,BOO004.A0101,Russia_Bolshoy,32.012301,153.249483,15,94.091290,5,74.916696,3,58.141296,...,83.982202,3475.0,LamnidisNatureCommunications2018,Russia_Bolshoy,2.730692,777001,1,all,silver,NaN
BOO005.A0101,BOO005.A0101,Russia_Bolshoy,23.379206,145.998818,14,102.499722,7,73.517417,4,44.828009,...,83.982202,3475.0,LamnidisNatureCommunications2018,Russia_Bolshoy,2.690273,717504,1,all,silver,NaN
BOO001.A0101,BOO001.A0101,Russia_Bolshoy,35.236603,180.272104,17,131.149401,8,102.846979,5,35.236603,...,83.982202,3745.0,LamnidisNatureCommunications2018,Russia_Bolshoy,1.126532,615153,1,all,silver,NaN
BOO003.A0101,BOO003.A0101,Russia_Bolshoy,18.153601,181.349129,23,92.981807,8,43.635599,3,0.000000,...,83.982202,3475.0,LamnidisNatureCommunications2018,Russia_Bolshoy,1.006430,609521,1,all,silver,NaN
BOO006.A0101,BOO006.A0101,Russia_Bolshoy,26.702786,113.137105,9,95.024791,5,76.246792,3,76.246792,...,83.982202,3475.0,LamnidisNatureCommunications2018,Russia_Bolshoy,0.995680,602052,1,all,silver,NaN
BOO002.A0101,BOO002.A0101,Russia_Bolshoy,13.262600,116.166475,16,63.287080,6,26.148099,2,0.000000,...,83.982202,3475.0,LamnidisNatureCommunications2018,Russia_Bolshoy,0.750120,537219,1,all,silver,NaN


In [4]:
df_ho = pd.read_csv("./Empirical/HO/combined_roh05.csv", sep="\t")

In [7]:
df_ho[df_ho["iid"].str.contains("Ju_hoan_North")]

,iid,pop,max_roh,sum_roh>4,n_roh>4,sum_roh>8,n_roh>8,sum_roh>12,n_roh>12,sum_roh>20,...,study,clst_alt,period_alt,include_alt,clst,mean_cov,med_cov,n_cov_snp_read,full_iid,n_cov_snp
407,Ju_hoan_North_4,Ju_hoan_North,13.635399,55.448484,8,24.811300,2,13.635399,1,0.0,...,Lazaridis et al. 2014,Ju_hoan_North,NaN,1,Ju_hoan_North,NaN,NaN,NaN,HGDP01036,551020
477,Ju_hoan_North_1,Ju_hoan_North,14.452398,45.814887,5,27.202696,2,27.202696,2,0.0,...,Lazaridis et al. 2014,Ju_hoan_North,NaN,1,Ju_hoan_North,NaN,NaN,NaN,HGDP00991,552765
500,Ju_hoan_North_2,Ju_hoan_North,17.709100,41.982499,5,17.709100,1,17.709100,1,0.0,...,Lazaridis et al. 2014,Ju_hoan_North,NaN,1,Ju_hoan_North,NaN,NaN,NaN,HGDP00992,552658
622,Ju_hoan_North_0,Ju_hoan_North,18.166298,27.997798,3,18.166298,1,18.166298,1,0.0,...,Lazaridis et al. 2014,Ju_hoan_North,NaN,1,Ju_hoan_North,NaN,NaN,NaN,HGDP00987,552123
820,Ju_hoan_North_3,Ju_hoan_North,9.546202,13.647597,2,9.546202,1,0.000000,0,0.0,...,Lazaridis et al. 2014,Ju_hoan_North,NaN,1,Ju_hoan_North,NaN,NaN,NaN,HGDP01032,551614


In [24]:
df_ho[df_ho["pop"].str.contains("Hadza")]

,iid,pop,max_roh,sum_roh>4,n_roh>4,sum_roh>8,n_roh>8,sum_roh>12,n_roh>12,sum_roh>20,...,study,clst_alt,period_alt,include_alt,clst,mean_cov,med_cov,n_cov_snp_read,full_iid,n_cov_snp
64,Hadza_3,Hadza,16.240204,264.950700,31,187.566915,17,68.000203,5,0.0,...,Lazaridis et al. 2014,Hadza,NaN,1,Hadza,NaN,NaN,NaN,End08,549841
203,Hadza_2,Hadza,18.397104,120.681022,18,38.452709,3,18.397104,1,0.0,...,Lazaridis et al. 2014,Hadza,NaN,1,Hadza,NaN,NaN,NaN,Bar13,540831
355,Hadza_4,Hadza,12.667095,66.326388,10,29.600291,3,12.667095,1,0.0,...,Lazaridis et al. 2014,Hadza,NaN,1,Hadza,NaN,NaN,NaN,Bar04,545031
743,Hadza_0,Hadza,8.227205,17.779204,3,8.227205,1,0.000000,0,0.0,...,Lazaridis et al. 2014,Hadza,NaN,1,Hadza,NaN,NaN,NaN,Bar08,543902
802,Hadza_1,Hadza,5.499005,14.886808,3,0.000000,0,0.000000,0,0.0,...,Lazaridis et al. 2014,Hadza,NaN,1,Hadza,NaN,NaN,NaN,Bar10,543967


In [ ]:
Hadza, Khomani, Ju_Huan_North